In [36]:
import pandas as pd
import datetime
import vk_api
import os
import requests
import json
import random

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [37]:
token = '8f188b33b9f55e51689fcf526eec08da643d3bb487bfdb366f1e42db49d51d6a7ad92d8495fa5b325b81f'

In [44]:
vk_session = vk_api.VkApi(token=token)

vk = vk_session.get_api()

In [52]:
help(vk.messages.send)

Help on VkApiMethod in module vk_api.vk_api object:

class VkApiMethod(builtins.object)
 |  VkApiMethod(vk, method=None)
 |  
 |  Дает возможность обращаться к методам API через:
 |  
 |  >>> vk = VkApiMethod(...)
 |  >>> vk.wall.getById(posts='...')
 |  или
 |  >>> vk.wall.get_by_id(posts='...')
 |  
 |  Methods defined here:
 |  
 |  __call__(self, **kwargs)
 |      Call self as a function.
 |  
 |  __getattr__(self, method)
 |  
 |  __init__(self, vk, method=None)
 |      Initialize self.  See help(type(self)) for accurate signature.



In [68]:
vk.messages.send(
    chat_id=1,
    random_id=random.randint(0, 10000),
    message='Sacha, hi hi')

8

In [65]:
vk.groups.getTokenPermissions()

{'mask': 134623237,
 'permissions': [{'name': 'photos', 'setting': 4},
  {'name': 'docs', 'setting': 131072},
  {'name': 'messages', 'setting': 4096},
  {'name': 'wall', 'setting': 8192},
  {'name': 'manage', 'setting': 262144}]}

In [69]:
df = pd.read_csv('/home/jupyter-l.fedoseeva-12/Lesson_7/ads_data.csv.zip', compression='zip')

In [70]:
ad_data = df.groupby(['ad_id', 'event'], as_index=False) \
    .agg({'user_id': 'count'})

In [71]:
ad_data = ad_data.pivot(index='ad_id', columns='event', values='user_id').reset_index()

In [72]:
ad_data = ad_data.fillna(0).assign(ctr=ad_data.click / ad_data.view)

In [73]:
top_ctr = ad_data.query('click > 20 & view > 100').sort_values('ctr').tail(10)

In [74]:
top_ctr.to_csv('top_ctr_data.csv', index=False)

In [76]:
path = '/home/jupyter-l.fedoseeva-12/Lesson_7/top_ctr_data.csv'
file_name = 'top_ctr_data.csv'

path_to_file = path
upload_url = vk.docs.getMessagesUploadServer(peer_id=2000000001)["upload_url"]
file = {'file': (file_name, open(path_to_file, 'rb'))}

response = requests.post(upload_url, files=file)

json_data = json.loads(response.text)
json_data

{'file': '-58445794|0|-1|536132|b50443b3cb|csv|403|top_ctr_data.csv|c2f9e346fc49058f43500d24dbf7ae44|05d82d8a4927845b3803882df9393e8c||||eyJkaXNrIjoiNDAiLCJwZWVyX3NlbmRlciI6Ii01ODQ0NTc5NCJ9'}

In [77]:
saved_file = vk.docs.save(file=json_data['file'], title=file_name)

In [78]:
saved_file

{'type': 'doc',
 'doc': {'id': 597951886,
  'owner_id': -58445794,
  'title': 'top_ctr_data.csv',
  'size': 403,
  'ext': 'csv',
  'date': 1622579458,
  'type': 1,
  'url': 'https://vk.com/doc-58445794_597951886?hash=fc4301c0082b7df21e&dl=FU2TQNBUGU3TSNA:1622579458:6fb3e5aa0fe7c37339&api=1&no_preview=1'}}

In [79]:
attachment = 'doc{}_{}'.format(saved_file['doc']['owner_id'], saved_file['doc']['id'])

In [80]:
attachment

'doc-58445794_597951886'

In [83]:
vk.messages.send(
    chat_id=1,
    random_id=4,
    message='Топ объявлений по CTR', 
    attachment = attachment
)

10